<a href="https://colab.research.google.com/github/jtnishi/Jupyter_Relative_Currency_Performance/blob/main/Relative_Currency_Performance_Information_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Relative Currency Performance Information Generator

**2022-10-22**

## Introduction

With the US Dollar reaching an exchange rate of ¥150 to $1, I wanted a way to be able to figure out what is the relative performance of these currencies against each other, to try to get a sense of how much of the exchange rate change is due to the strengthening of one currency relative to others, and how much is due to the weakening of another currency relative to others.  In my 
[original Google spreadsheet](https://docs.google.com/spreadsheets/d/11huoUDaFcvoWdmGiiSBFhur1t821qRrtlLUv9gt5xpY/edit?usp=sharing) for this idea, I used the Swiss Franc as the reference currency.  
  
This spreadsheet works, but it is kind of inelegant in how it looks and functions. I'd rather have some code do the work to output the data and give me graphs.  Ergo, let's do this with some Python.  
  
---


## Methodology

I will start with a reference date and a reference currency.  That will effectively act as a time zero.  We will calculate the conversion of each currency compared to the reference currency.  We will take that as the 100% point of the data.  We can then take the conversion of the currency from each day going forward, and calculate it relative to the original conversion.  That will give us a relative performance of each currency compared to the reference.

Ideally, we want to pick a reference currency that kind of reflects something like a "world average".  Google Finance doesn't support the IMF's Special Drawing Rights (XDR).  So I originally picked the Swiss Franc (CHF) as a currency that should be neutral-ish and not super involved in the politics of the world, due to Switzerland being mostly neutral famously.  
  
With this code, I can actually use any API that I can access with Python code, so I may instead choose to use a different data source.  

For this case, we'll use [this api](https://github.com/fawazahmed0/currency-api#readme) as our base, because it's a simple one that just basically just pulls the data out of a CDN.  This works just fine.

---

---
---
---

## Execution

For those of you that haven't dealt with Jupyter Notebooks or Google Colab, this is what you need to do:

1. Upper right hand corner, click on the triangle menu, and click "Connect to a hosted runtime."
2. For each section, follow the instructions in each section above if any to make modifications, and then hit the play button next to each chunk of Python code.
3. Go one step at a time, waiting until each segment finishes. You'll see a checkmark after each one.  For example, the next piece of code, if it works should just output:
```
    Congratulations, you've run this successfully.
```

In [18]:
print("Congratulations, you've run this successfully")

Congratulations, you've run this successfully


---

### Step 0 - RUN THIS FIRST

Here, we're going to define some basic parts of the Python code.  This will cover imports, some basic utility functions, and some constant definitons.

In [19]:
##########################
#  === PIP INSTALLS ===  #
##########################

# For graph stuff.
!pip install --upgrade plotly

#####################
#  === IMPORTS ===  #
#####################

from datetime import date as DTDate, timedelta as DTDelta
import json
import logging
import pprint

import pandas as pd
import requests
import tabulate

#######################
#  === CONSTANTS ===  #
#######################

# https://github.com/fawazahmed0/currency-api#readme
API_URL_FORMAT = 'https://cdn.jsdelivr.net/gh/fawazahmed0/currency-api@{apiVersion}/{date}{endpoint}.json'

# Per the documentation, the API version hasn't really changed.
API_VERSION = 1

####################
#  === LOGGER ===  #
####################

logging.basicConfig(level=logging.INFO, force=True)
#logging.basicConfig(level=logging.DEBUG, force=True)
LOG = logging.getLogger('log')
LOG.setLevel(logging.INFO)
#LOG.setLevel(logging.DEBUG)

################################################################################
################################################################################
################################################################################


def log_obj(obj_to_log, title='Logged Object', log_func=LOG.debug):
  """
  Log the object sent to the function to the system.  Useful for checking the
  format of data.
  """

  log_func('%s:' % title)
  log_func('')
  for line in pprint.pformat(obj_to_log).split('\n'):
    log_func('  %s' % line)


def get_api_data(endpoint, date='latest', api_version=API_VERSION):
  """
  Get the data out of the API, given dates and endpoint.
  """
  if isinstance(date, DTDate):
    date = date.isoformat()

  data_url = API_URL_FORMAT.format(apiVersion=api_version, date=date, endpoint=endpoint)
  LOG.debug('API Data Call to: %s', data_url)

  try:
    response = requests.get(data_url)
    response.raise_for_status()
    response_body = response.json()
    LOG.debug('API Data Call to %s response code: %d', data_url, response.status_code)
    log_obj(response_body, 'Response to %s' % data_url, log_func=LOG.debug)
    return response_body

  except requests.HTTPError as exc:
    LOG.error("Error in retrieving data. Exception: %s" % exc)
    return None


def get_currencies():
  """
  Gets the list of currencies from the API.
  """
  return get_api_data('/currencies.min')


def currency_data_mapping(currencies):
  """
  Maps all currency names to the full name.
  """
  # Convert singleton currency to list.
  if not isinstance(currencies, list):
    currencies = [currencies]

  ret_data = {'BAD_CURRENCIES': []}
  valid_currencies = get_currencies()

  for currency in currencies:
    if currency in valid_currencies:
      ret_data[currency] = valid_currencies[currency]
    else:
      ret_data['BAD_CURRENCIES'].append(currency)
  
  return ret_data


def currencies_in_data(currencies):
  """
  Validates that each of the currencies are in the data set. Throws an
  exception if a currency is not in the set.
  """

  currency_map = currency_data_mapping(currencies)

  # Finally, return information and error as needed.
  if not currency_map['BAD_CURRENCIES']:
    LOG.debug('All currencies found: %s', currencies)
    del currency_map['BAD_CURRENCIES']
    return currency_map

  errmsg = 'Currencies not found in set: %s' % pprint.pformat(currency_map['BAD_CURRENCIES'])
  raise ValueError(errmsg)


def get_date_ranges(start_date, end_date=DTDate.today()):
  """
  Generates the list of dates we will need to use to get information out of
  the API.
  """
  num_days = (end_date - start_date).days + 1
  dates = [(start_date + DTDelta(days=i)).isoformat()
           for i in range(num_days)]
  return dates

  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


---

### *Optional - Reference - All Valid Currencies and their Symbols*

Before we start, just to make sure we have a tabular list of all available currencies to convert from, you can run the following cell to generate a table of all the currencies in the reference data list from the API.  This will be used in the configuration step below.

**NOTE: This includes a mish-mash of both cryptocurrencies AND regular currencies.  You will want to filter accordingly.**

In [20]:
def print_currency_table(data):
  if data:
    table_items = [[symbol, name] for symbol, name in data.items()]
    return tabulate.tabulate(table_items, headers=('Symbol', 'Currency Name'), tablefmt='simple')
  return None

data = get_currencies()
formatted = print_currency_table(data)

print('Table of currencies:\n')
print(formatted)

Table of currencies:

Symbol    Currency Name
--------  --------------------------------------
1inch     1inch Network
aave      Aave
ada       Cardano
aed       United Arab Emirates Dirham
afn       Afghan afghani
algo      Algorand
all       Albanian lek
amd       Armenian dram
amp       Synereo
ang       Netherlands Antillean Guilder
aoa       Angolan kwanza
ar        Arweave
ars       Argentine peso
atom      Atomic Coin
aud       Australian dollar
avax      Avalanche
awg       Aruban florin
axs       AXS
azn       Azerbaijani manat
bam       Bosnia-Herzegovina Convertible Mark
bat       Basic Attention Token
bbd       Bajan dollar
bch       Bitcoin Cash
bdt       Bangladeshi taka
bgn       Bulgarian lev
bhd       Bahraini dinar
bif       Burundian Franc
bmd       Bermudan dollar
bnb       Binance Coin
bnd       Brunei dollar
bob       Bolivian boliviano
brl       Brazilian real
bsd       Bahamian dollar
bsv       Bitcoin SV
btc       Bitcoin
btcb      Bitcoin BEP2
btg       Bitcoi

---

### Step 1. Configuration.

Use the next box to configure the parameters to be used by the executing code. This will be used to make the right calls to the finance APIs.  You can fill in the form as needed.

In [21]:
###########################
#  === CONFIGURATION ===  #
###########################

# Put the symbol of the reference currency that all other currencies will be
# compared against.  For the table of all possible symbols, use the data from
# the cell above.
REFERENCE_CURRENCY = 'chf'  # Swiss Franc
#REFERENCE_CURRENCY = 'xdr'  # IMF Special Drawing Rights

# Create the list of currencies to convert relative to the reference currency.
CONVERSION_CURRENCIES = [
    'usd',  # United States dollar
    'jpy',  # Japanese Yen
    'eur',  # Euro
    'gbp',  # Pound Sterling
    'cny',  # Chinese Yuan
    'inr',  # Indian Rupee
    'cad',  # Canadian Dollar
    'aud',  # Australian Dollar
    'mxn',  # Mexican Peso
    'chf',  # Swiss Franc
    'sek',  # Swedish Krona
    'nzd',  # New Zealand Dollar
    'rub',  # Russian Ruble
    'hkd',  # Hong Kong Dollar
    'zar',  # South African Rand
    'xdr',  # IMF Special Drawing Rights

    # Some cryptos mostly for (making) fun (of).
    'btc',  # Bitcoin
    'eth',  # Ethereum
    'doge',  # Dogecoin
]

START_DATE = DTDate.fromisoformat('2022-01-01')
END_DATE = DTDate.today()


#### *Optional - Data Sanity Check*

The following cell runs a sanity check on the configuration data above and logs the information to the notebook.

In [22]:
# Sanity check of data

def step_01_config_sanity_check(currency_list, start_date, end_date):
  """
  Runs a quick sanity check of the data to make sure that
  the config looks sound.
  """
  mapping = currencies_in_data(currency_list)
  log_obj(mapping, "Currency Mapping", log_func=LOG.info)
  dates = get_date_ranges(start_date, end_date)
  LOG.info('Number of dates: %d', len(dates))
  LOG.info('First Date: %s', dates[0])
  LOG.info('Last Date: %s', dates[-1])
  log_obj(dates, "Dates in Range", log_func=LOG.debug)

step_01_config_sanity_check(
    [REFERENCE_CURRENCY] + CONVERSION_CURRENCIES,
    START_DATE,
    END_DATE
)

INFO:log:Currency Mapping:
INFO:log:
INFO:log:  {'aud': 'Australian dollar',
INFO:log:   'btc': 'Bitcoin',
INFO:log:   'cad': 'Canadian dollar',
INFO:log:   'chf': 'Swiss franc',
INFO:log:   'cny': 'Chinese Yuan',
INFO:log:   'doge': 'Dogecoin',
INFO:log:   'eth': 'Ether',
INFO:log:   'eur': 'Euro',
INFO:log:   'gbp': 'Pound sterling',
INFO:log:   'hkd': 'Hong Kong dollar',
INFO:log:   'inr': 'Indian rupee',
INFO:log:   'jpy': 'Japanese yen',
INFO:log:   'mxn': 'Mexican peso',
INFO:log:   'nzd': 'New Zealand dollar',
INFO:log:   'rub': 'Russian ruble',
INFO:log:   'sek': 'Swedish krona',
INFO:log:   'usd': 'United States dollar',
INFO:log:   'xdr': 'Special Drawing Rights',
INFO:log:   'zar': 'South African rand'}
INFO:log:Number of dates: 294
INFO:log:First Date: 2022-01-01
INFO:log:Last Date: 2022-10-21


---

### Step 2: Run through the API and gather data

This does the hard work step of going through the API, gathering up the data from the individual date records, and compiling them into a couple of data frames that will represent the actual financial data.

In [23]:
def data_for_date(reference_currency, date, conversion_currencies):
  """
  For a given date and reference currency, get the conversion data for the
  currencies.
  """
  if not isinstance(conversion_currencies, list):
    conversion_currencies = [conversion_currencies]

  endpoint = f'/currencies/{reference_currency}'
  data = get_api_data(endpoint, date)

  if not data:
    data = {}

  data_date = data.get('date')
  LOG.debug('Captured date: %s', data_date)

  if reference_currency in data:
    data = data[reference_currency]
  else:
    data = {}

  if not data:
    LOG.warning('Data missing for reference currency %s for date %s',
                reference_currency,
                date)
    return {currency: None for currency in conversion_currencies}

  ret_data = {}
  for currency in conversion_currencies:
    if currency in data:
      # NOTE: When we do the power relationship, we need to use the INVERSE
      # of each of the currencies.  That is, convert 1 of the base currency to
      # our reference.  For now, we'll just store the raw data.  That's useful
      # to refer to from the beginning
      ret_data[currency] = data[currency]

    else:
      LOG.warning('Data missing for currency %s in data for base currency %s '
                  'for date %s',
                  currency, reference_currency, date)
      ret_data[currency] = None

  return ret_data


def data_for_dates(reference_currency, start_date, end_date, conversion_currencies):
  """
  Get the currency conversion data for a full set of dates.
  """
  dates = get_date_ranges(start_date, end_date)
  
  # @TODO: Parallelize this!
  ret_data = {}
  len_dates = len(dates)

  LOG.info('Retrieving data for %d dates.  This will take a while.', len_dates)
  for idx, date in enumerate(dates):
    LOG.debug('Getting data for %s', date)
    ret_data[date] = data_for_date(reference_currency, date, conversion_currencies)

    cur_cnt = idx + 1
    if cur_cnt % 10 == 0:
      LOG.info('Finished %d of %d dates.', cur_cnt, len_dates)

  LOG.info('Done retrieving information for all dates.')
  ret_pd = pd.DataFrame.from_dict(ret_data)

  return ret_pd


def inverted_conversions(currency_data):
  """
  For graphing relative power of each currency, we actually need the inverse of
  the input.  That is, we need to convert 1 unit of the currencies we want as
  our lookups to our reference currency.  The base table gives us the opposite,
  one unit of the reference currency in the others.  However, doing the call
  this way does far fewer API calls for our API.
  """
  return currency_data.pow(-1)


def relative_currency_power(inverse_currency_data):
  """
  Finally, calculate the relative value of each currency to the reference
  currency on the initial date.
  """
  first_date = inverse_currency_data.columns[0]
  base_data = inverse_currency_data[first_date]
  relative_power_data = inverse_currency_data.div(base_data, axis=0)
  return relative_power_data


def remap_data(relative_data):
  """
  For final presentation, convert the labels of the base currencies to the
  full labels of the currencies.
  """
  currencies = list(relative_data.index)
  currency_map = currency_data_mapping(currencies)
  final_map = {
      currency: '%s (%s)' % (name, currency.upper())
      for currency, name in currency_map.items()
      if currency != 'BAD_CURRENCIES'
  }
  renamed_data = relative_data.rename(mapper=final_map, axis=0)
  return renamed_data

###############################################################################

def step_02_full_data_process(reference_currency, start_date, end_date, conversion_currencies):
  """
  Handle all the processing of step 2.
  """
  base_currency_data = data_for_dates(reference_currency, start_date, end_date, conversion_currencies)
  inverted_bc_data = inverted_conversions(base_currency_data)
  relative_bc_data = relative_currency_power(inverted_bc_data)
  renamed_data = remap_data(relative_bc_data)

  ret_data = {
      'data': renamed_data,      
      'label': 'Relative Currency Strength',
      'reference': {
        'Raw Currency Data': base_currency_data,
        'Inverse Conversion Data (X->Ref)': inverted_bc_data,
        'Relative Conversion Data': relative_bc_data,
      }
  }

  return ret_data

###############################################################################

step_02_data = step_02_full_data_process(REFERENCE_CURRENCY, START_DATE, END_DATE, CONVERSION_CURRENCIES)



INFO:log:Retrieving data for 294 dates.  This will take a while.
INFO:log:Finished 10 of 294 dates.
INFO:log:Finished 20 of 294 dates.
INFO:log:Finished 30 of 294 dates.
INFO:log:Finished 40 of 294 dates.
INFO:log:Finished 50 of 294 dates.
INFO:log:Finished 60 of 294 dates.
INFO:log:Finished 70 of 294 dates.
INFO:log:Finished 80 of 294 dates.
INFO:log:Finished 90 of 294 dates.
INFO:log:Finished 100 of 294 dates.
INFO:log:Finished 110 of 294 dates.
INFO:log:Finished 120 of 294 dates.
ERROR:log:Error in retrieving data. Exception: 403 Client Error: Forbidden for url: https://cdn.jsdelivr.net/gh/fawazahmed0/currency-api@1/2022-05-01/currencies/chf.json
INFO:log:Finished 130 of 294 dates.
INFO:log:Finished 140 of 294 dates.
INFO:log:Finished 150 of 294 dates.
INFO:log:Finished 160 of 294 dates.
INFO:log:Finished 170 of 294 dates.
INFO:log:Finished 180 of 294 dates.
INFO:log:Finished 190 of 294 dates.
INFO:log:Finished 200 of 294 dates.
INFO:log:Finished 210 of 294 dates.
INFO:log:Finished 

#### *Optional - Data Table*

This shows the core data gathered after the retrieval phase

In [24]:
step_02_data['data']

,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,2022-01-10,...,2022-10-12,2022-10-13,2022-10-14,2022-10-15,2022-10-16,2022-10-17,2022-10-18,2022-10-19,2022-10-20,2022-10-21
United States dollar (USD),1.0,1.000000,1.000937,1.006708,1.004674,1.006281,1.010711,1.007882,1.007567,1.008102,...,1.094695,1.093163,1.096122,1.103802,1.103802,1.101261,1.091604,1.090482,1.102638,1.102276
Japanese yen (JPY),1.0,1.000000,0.999995,1.004606,0.995785,0.997481,1.004048,1.003445,1.003382,1.003303,...,0.862320,0.857200,0.855866,0.853961,0.853961,0.853047,0.844302,0.841453,0.846884,0.844713
Euro (EUR),1.0,1.000000,1.000646,1.000961,0.996960,1.001149,1.003840,1.007017,1.006702,1.006391,...,0.933638,0.933807,0.942830,0.943827,0.943827,0.943292,0.944532,0.945710,0.946767,0.947260
Pound sterling (GBP),1.0,1.000856,1.001216,1.003524,1.005630,1.009008,1.011614,1.013479,1.012677,1.013444,...,0.887380,0.898591,0.917435,0.912072,0.912072,0.914585,0.916828,0.914846,0.914280,0.913866
Chinese Yuan (CNY),1.0,1.000000,1.000938,1.006677,1.004768,1.006263,1.006423,1.004437,1.004123,1.004656,...,0.970984,0.968394,0.971004,0.975612,0.975612,0.973373,0.964098,0.962358,0.969492,0.970772
Indian rupee (INR),1.0,1.000000,1.000937,1.008204,1.004389,1.007412,1.012012,1.008512,1.010775,1.011312,...,0.991450,0.990424,0.992246,0.997944,0.997944,0.993312,0.989440,0.987476,0.989910,0.991653
Canadian dollar (CAD),1.0,1.000000,0.999988,0.997984,0.999565,0.996790,1.003692,1.007803,1.007049,1.007389,...,1.002531,1.000539,1.009566,1.005060,1.005060,1.004926,1.006012,1.004158,1.011843,1.011037
Australian dollar (AUD),1.0,1.000000,1.001596,0.996992,1.001076,0.999774,0.995792,0.995687,0.995161,0.995772,...,0.941920,0.945244,0.954542,0.941335,0.941335,0.942713,0.946505,0.947281,0.949603,0.950884
Mexican peso (MXN),1.0,1.000000,1.001433,1.007141,1.002859,1.002399,1.010106,1.013117,1.013376,1.013010,...,1.117754,1.121022,1.124331,1.127037,1.127037,1.126666,1.119756,1.116585,1.122881,1.127121
Swiss franc (CHF),1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


#### *Optional - Raw Currency Data*

The raw currency data for the reference currency converted to each of the targeted currencies is below.

In [25]:
step_02_data['reference']['Raw Currency Data']

,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,2022-01-10,...,2022-10-12,2022-10-13,2022-10-14,2022-10-15,2022-10-16,2022-10-17,2022-10-18,2022-10-19,2022-10-20,2022-10-21
usd,1.096847,1.096847,1.095820,1.089538,1.091744,1.090001,1.085223,1.088269,1.088610,1.088032,...,1.001966,1.003370,1.000661,0.993699,0.993699,0.995992,1.004803,1.005837,0.994748,0.995075
jpy,126.255937,126.255937,126.256630,125.677057,126.790328,126.574799,125.746890,125.822470,125.830322,125.840230,...,146.414223,147.288787,147.518399,147.847418,147.847418,148.005778,149.538804,150.045199,149.082857,149.466155
eur,0.964562,0.964562,0.963939,0.963636,0.967503,0.963455,0.960872,0.957841,0.958141,0.958437,...,1.033122,1.032935,1.023050,1.021969,1.021969,1.022549,1.021206,1.019934,1.018796,1.018265
gbp,0.811282,0.810588,0.810297,0.808433,0.806740,0.804039,0.801968,0.800492,0.801126,0.800520,...,0.914244,0.902838,0.884294,0.889493,0.889493,0.887049,0.884879,0.886796,0.887345,0.887747
cny,6.971456,6.971456,6.964926,6.925215,6.938371,6.928066,6.926965,6.940659,6.942833,6.939144,...,7.179785,7.198989,7.179637,7.145724,7.145724,7.162166,7.231062,7.244140,7.190834,7.181352
inr,81.730531,81.730531,81.654043,81.065477,81.373365,81.129198,80.760471,81.040746,80.859251,80.816349,...,82.435380,82.520778,82.369224,81.898916,81.898916,82.280832,82.602845,82.767140,82.563614,82.418480
cad,1.386322,1.386322,1.386339,1.389122,1.386925,1.390787,1.381222,1.375588,1.376618,1.376154,...,1.382822,1.385575,1.373186,1.379343,1.379343,1.379527,1.378037,1.380581,1.370096,1.371188
aud,1.509043,1.509043,1.506639,1.513596,1.507421,1.509384,1.515420,1.515580,1.516381,1.515450,...,1.602093,1.596459,1.580908,1.603088,1.603088,1.600745,1.594331,1.593026,1.589130,1.586990
mxn,22.482411,22.482411,22.450250,22.323011,22.418325,22.428608,22.257478,22.191338,22.185663,22.193671,...,20.113911,20.055275,19.996260,19.948242,19.948242,19.954818,20.077958,20.134971,20.022086,19.946766
chf,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


#### *Optional - Inverse Conversion Data*

The data for the inverse conversion (ie: converting 1 unit of each currency to our reference currency), done by literally taking 1/x for each cell.

In [26]:
step_02_data['reference']['Inverse Conversion Data (X->Ref)']

,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,2022-01-10,...,2022-10-12,2022-10-13,2022-10-14,2022-10-15,2022-10-16,2022-10-17,2022-10-18,2022-10-19,2022-10-20,2022-10-21
usd,0.911704,0.911704,0.912559,0.917820,0.915966,0.917430,0.921470,0.918890,0.918603,0.919091,...,0.998038,0.996641,0.999339,1.006341,1.006341,1.004024,0.995220,0.994197,1.005280,1.004949
jpy,0.007920,0.007920,0.007920,0.007957,0.007887,0.007900,0.007952,0.007948,0.007947,0.007947,...,0.006830,0.006789,0.006779,0.006764,0.006764,0.006756,0.006687,0.006665,0.006708,0.006690
eur,1.036740,1.036740,1.037410,1.037736,1.033589,1.037931,1.040721,1.044015,1.043688,1.043365,...,0.967940,0.968115,0.977469,0.978503,0.978503,0.977948,0.979234,0.980456,0.981551,0.982063
gbp,1.232617,1.233672,1.234115,1.236961,1.239557,1.243721,1.246933,1.249232,1.248243,1.249188,...,1.093800,1.107618,1.130846,1.124236,1.124236,1.127333,1.130098,1.127655,1.126957,1.126447
cny,0.143442,0.143442,0.143577,0.144400,0.144126,0.144340,0.144363,0.144079,0.144033,0.144110,...,0.139280,0.138908,0.139283,0.139944,0.139944,0.139623,0.138292,0.138043,0.139066,0.139250
inr,0.012235,0.012235,0.012247,0.012336,0.012289,0.012326,0.012382,0.012339,0.012367,0.012374,...,0.012131,0.012118,0.012140,0.012210,0.012210,0.012153,0.012106,0.012082,0.012112,0.012133
cad,0.721333,0.721333,0.721324,0.719879,0.721020,0.719017,0.723997,0.726962,0.726418,0.726663,...,0.723159,0.721722,0.728233,0.724983,0.724983,0.724886,0.725670,0.724333,0.729876,0.729295
aud,0.662672,0.662672,0.663729,0.660678,0.663385,0.662522,0.659883,0.659813,0.659465,0.659870,...,0.624183,0.626386,0.632548,0.623796,0.623796,0.624709,0.627222,0.627736,0.629275,0.630124
mxn,0.044479,0.044479,0.044543,0.044797,0.044606,0.044586,0.044929,0.045063,0.045074,0.045058,...,0.049717,0.049862,0.050009,0.050130,0.050130,0.050113,0.049806,0.049665,0.049945,0.050133
chf,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


---

### Step 3: Visualize the Core data

Now that we have the data calculated in step 2 above, we can send the data to be visualized.

For this we'll use one of the multitude of graphing libraries appropriate for this.

In [27]:
# Starting Data
step_02_data['data']

,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,2022-01-10,...,2022-10-12,2022-10-13,2022-10-14,2022-10-15,2022-10-16,2022-10-17,2022-10-18,2022-10-19,2022-10-20,2022-10-21
United States dollar (USD),1.0,1.000000,1.000937,1.006708,1.004674,1.006281,1.010711,1.007882,1.007567,1.008102,...,1.094695,1.093163,1.096122,1.103802,1.103802,1.101261,1.091604,1.090482,1.102638,1.102276
Japanese yen (JPY),1.0,1.000000,0.999995,1.004606,0.995785,0.997481,1.004048,1.003445,1.003382,1.003303,...,0.862320,0.857200,0.855866,0.853961,0.853961,0.853047,0.844302,0.841453,0.846884,0.844713
Euro (EUR),1.0,1.000000,1.000646,1.000961,0.996960,1.001149,1.003840,1.007017,1.006702,1.006391,...,0.933638,0.933807,0.942830,0.943827,0.943827,0.943292,0.944532,0.945710,0.946767,0.947260
Pound sterling (GBP),1.0,1.000856,1.001216,1.003524,1.005630,1.009008,1.011614,1.013479,1.012677,1.013444,...,0.887380,0.898591,0.917435,0.912072,0.912072,0.914585,0.916828,0.914846,0.914280,0.913866
Chinese Yuan (CNY),1.0,1.000000,1.000938,1.006677,1.004768,1.006263,1.006423,1.004437,1.004123,1.004656,...,0.970984,0.968394,0.971004,0.975612,0.975612,0.973373,0.964098,0.962358,0.969492,0.970772
Indian rupee (INR),1.0,1.000000,1.000937,1.008204,1.004389,1.007412,1.012012,1.008512,1.010775,1.011312,...,0.991450,0.990424,0.992246,0.997944,0.997944,0.993312,0.989440,0.987476,0.989910,0.991653
Canadian dollar (CAD),1.0,1.000000,0.999988,0.997984,0.999565,0.996790,1.003692,1.007803,1.007049,1.007389,...,1.002531,1.000539,1.009566,1.005060,1.005060,1.004926,1.006012,1.004158,1.011843,1.011037
Australian dollar (AUD),1.0,1.000000,1.001596,0.996992,1.001076,0.999774,0.995792,0.995687,0.995161,0.995772,...,0.941920,0.945244,0.954542,0.941335,0.941335,0.942713,0.946505,0.947281,0.949603,0.950884
Mexican peso (MXN),1.0,1.000000,1.001433,1.007141,1.002859,1.002399,1.010106,1.013117,1.013376,1.013010,...,1.117754,1.121022,1.124331,1.127037,1.127037,1.126666,1.119756,1.116585,1.122881,1.127121
Swiss franc (CHF),1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Let's turn the above data now into a line plot.

In [28]:
import plotly.express as px

def fix_data_table(data_table):
  """
  We need to clean up the data table a bit.

  This involves un-pivoting the data, and renaming to something useful for the graph.
  """
  # Convert the data into long format data
  modded_data = data_table.reset_index()
  melted = modded_data.melt(
      id_vars='index',
      var_name='date',
      value_name='value'
  )

  # And rename the columns
  output_column_names = {
    'index': 'Currency',
    'date': 'Date',
    'value': 'Relative Currency Value'
  }

  melted.rename(columns=output_column_names, inplace=True)
  return melted


def generate_title(ref_currency, start, end):
  """
  Get the title name for our graph.
  """
  return (f'Relative Value of Currencies compared to {ref_currency} from '
          f'{start} to {end}')


###############################################################################

def step_03_visualize(data, ref_currency, start, end):
  """
  Handle the actual visualization generation.
  """
  title = generate_title(ref_currency, start, end)
  fixed_data = fix_data_table(data)
  figure = px.line(
    fixed_data,
    x='Date',
    y='Relative Currency Value',
    color='Currency',
    title=title
  )
  figure.show()


###############################################################################

# And run!
step_03_visualize(step_02_data['data'], REFERENCE_CURRENCY, START_DATE, END_DATE)

## DONE!

Thanks for reading!